## Custom GPT Model - Shakespear Dataset

In [ ]:
#This is for FIRST RUN ONLY to download shakespear data

'''
import wget
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
shakes = wget.download(url)
shakes

In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("Length of dataset: " + str(len(text)))

Length of dataset: 1115394


In [3]:
print(text[:500]) #Print first 500 characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
# Unique characters in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### 1. Encoder - Decoder

In [5]:
#Create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] #Encoder: Take string input and produce integer
decode = lambda l: ''.join([itos[i] for i in l]) #Encoder: Take list of integers and produce string

print(encode("Hello meesam"))
print(decode(encode("Hello meesam")))

#It should be noted that our tokenizer is CHARACTER LEVEL tokenizer
#GPT and GOOGle use BPE (byte pair encoding) or other schemes - subword basis

[20, 43, 50, 50, 53, 1, 51, 43, 43, 57, 39, 51]
Hello meesam


### 2. Torch tensors

In [6]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### 3. Train / validation split

In [7]:
n = int(0.9*len(data))  #90% is for training rest validation
train_data = data[:n]
val_data = data[n:]

In [8]:
#Let's do training on data CHUNKS
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"When input is {context} then target is : {target}")

When input is tensor([18]) then target is : 47
When input is tensor([18, 47]) then target is : 56
When input is tensor([18, 47, 56]) then target is : 57
When input is tensor([18, 47, 56, 57]) then target is : 58
When input is tensor([18, 47, 56, 57, 58]) then target is : 1
When input is tensor([18, 47, 56, 57, 58,  1]) then target is : 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) then target is : 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is : 58


### 4. Batch of Input Tensors

In [13]:
torch.manual_seed(1337)
batch_size = 4              # number of independent sequences to be processed in parallel
block_size = 8              # maximum context length of predictions
def get_batch(split):
    #Generate small batch of inputs of data input x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [19]:
xb, yb = get_batch('train')
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)
print('-------------')
for b in range(batch_size):          #Batch dimension
    for t in range(block_size):      #Time dimension
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target is: {target}")


inputs: 
torch.Size([4, 8])
tensor([[52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
        [39, 49, 12,  1, 27,  1, 58, 56]])
targets: 
torch.Size([4, 8])
tensor([[42,  8,  0,  0, 23, 21, 26, 19],
        [53, 42, 57,  0, 23, 43, 43, 54],
        [ 1, 61, 39, 57,  1, 51, 53, 56],
        [49, 12,  1, 27,  1, 58, 56, 39]])
-------------
When input is [52] the target is: 42
When input is [52, 42] the target is: 8
When input is [52, 42, 8] the target is: 0
When input is [52, 42, 8, 0] the target is: 0
When input is [52, 42, 8, 0, 0] the target is: 23
When input is [52, 42, 8, 0, 0, 23] the target is: 21
When input is [52, 42, 8, 0, 0, 23, 21] the target is: 26
When input is [52, 42, 8, 0, 0, 23, 21, 26] the target is: 19
When input is [45] the target is: 53
When input is [45, 53] the target is: 42
When input is [45, 53, 42] the target is: 57
When input is [45, 53, 42, 57] the target is: 0
When input is [45, 53, 42, 57,

In [20]:
print(xb)

tensor([[52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
        [39, 49, 12,  1, 27,  1, 58, 56]])


### 5. Feed to BigramLanguageModel Neural Network (NN)

In [21]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #Each token reads off directly reads off the logits for the next token lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        #idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx)
        return logits

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


ValueError: too many values to unpack (expected 2)